# Matching - Bout IDs

In [1]:
# standard library imports
import os

# third party imports
import pandas as pd

# local imports


data_dir = os.path.join(os.path.dirname("__file__"), "..", "..", "data")
clean_data_dir = os.path.join(data_dir, "clean")

In [2]:
# Filter for only UFC events (so exclude WEC, Pride, etc. that were merged into UFC later)
ufcstats_events = pd.read_csv(os.path.join(clean_data_dir, "UFC Stats", "events.csv"))
event_ids = ufcstats_events.loc[ufcstats_events["is_ufc_event"] == 1, "id"].values

# Subset UFC-only bouts
ufcstats_bouts = pd.read_csv(os.path.join(clean_data_dir, "UFC Stats", "bouts.csv"))
ufcstats_bouts = ufcstats_bouts.loc[
    ufcstats_bouts["event_id"].isin(event_ids)
].reset_index(drop=True)

# Load in mapping tables
events_linkage = pd.read_csv(os.path.join(clean_data_dir, "event_mapping.csv"))
fighters_linkage = pd.read_csv(os.path.join(clean_data_dir, "fighter_mapping.csv"))

## Tapology

In [3]:
tapology_bouts = pd.read_csv(os.path.join(clean_data_dir, "Tapology", "bouts.csv"))

# First 345 fights need to be matched, otherwise already matched sequentially for us
temp1 = tapology_bouts.iloc[345:]
temp2 = ufcstats_bouts.iloc[345:]
all(temp2["id"].values == temp1["ufcstats_id"].values)  # type: ignore

True

In [4]:
temp_match1 = pd.DataFrame(
    {
        "ufcstats_id": ufcstats_bouts.loc[345:, "id"].values,
        "tapology_id": tapology_bouts.loc[345:, "id"].values,
    }
)

# Deal with rest
ufcstats_bouts2 = ufcstats_bouts.loc[:344].copy()
tapology_bouts2 = tapology_bouts.loc[:344].copy()
tapology_bouts2 = tapology_bouts2[
    ["id", "event_id", "bout_order", "fighter_1_id", "fighter_2_id"]
]

# Link event and fighter mappings
tapology_bouts2_temp = tapology_bouts2.merge(
    events_linkage[["ufcstats_id", "tapology_id"]].rename(
        columns={"ufcstats_id": "ufcstats_event_id", "tapology_id": "event_id"}
    ),
    how="left",
    on="event_id",
).drop(columns=["event_id"])
tapology_bouts2_temp = tapology_bouts2_temp.merge(
    fighters_linkage[["ufcstats_id", "tapology_id"]].rename(
        columns={"ufcstats_id": "ufcstats_fighter_1_id", "tapology_id": "fighter_1_id"}
    ),
    how="left",
    on="fighter_1_id",
).drop(columns=["fighter_1_id"])
tapology_bouts2_temp = tapology_bouts2_temp.merge(
    fighters_linkage[["ufcstats_id", "tapology_id"]].rename(
        columns={"ufcstats_id": "ufcstats_fighter_2_id", "tapology_id": "fighter_2_id"}
    ),
    how="left",
    on="fighter_2_id",
).drop(columns=["fighter_2_id"])
tapology_bouts2_temp = tapology_bouts2_temp.rename(columns={"id": "tapology_id"})

# Match on fighter IDs, bout ID, and bout order within event
temp_match2 = ufcstats_bouts2.merge(
    tapology_bouts2_temp,
    how="inner",
    left_on=["event_id", "bout_order", "red_fighter_id", "blue_fighter_id"],
    right_on=[
        "ufcstats_event_id",
        "bout_order",
        "ufcstats_fighter_1_id",
        "ufcstats_fighter_2_id",
    ],
)[["id", "tapology_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

matching1 = pd.concat([temp_match2, temp_match1], ignore_index=True).reset_index(
    drop=True
)

## ESPN

In [5]:
espn_bouts = pd.read_csv(os.path.join(clean_data_dir, "ESPN", "bouts.csv")).drop(
    columns=["winner_id", "card_segment"]
)

# Merge with events and bouts mappings
espn_bouts = espn_bouts.merge(
    events_linkage[["ufcstats_id", "espn_id"]].rename(
        columns={"ufcstats_id": "ufcstats_event_id", "espn_id": "event_id"}
    ),
    how="left",
    on="event_id",
).drop(columns=["event_id"])
espn_bouts = espn_bouts.merge(
    fighters_linkage[["ufcstats_id", "espn_id"]].rename(
        columns={"ufcstats_id": "ufcstats_fighter_1_id", "espn_id": "fighter_1_id"}
    ),
    how="left",
    on="fighter_1_id",
).drop(columns=["fighter_1_id"])
espn_bouts = espn_bouts.merge(
    fighters_linkage[["ufcstats_id", "espn_id"]].rename(
        columns={"ufcstats_id": "ufcstats_fighter_2_id", "espn_id": "fighter_2_id"}
    ),
    how="left",
    on="fighter_2_id",
).drop(columns=["fighter_2_id"])
espn_bouts = espn_bouts.rename(columns={"id": "espn_id"})

# Match on bout order, event ID, and fighter IDs
temp_match1 = ufcstats_bouts.merge(
    espn_bouts,
    how="inner",
    left_on=["event_id", "bout_order", "red_fighter_id", "blue_fighter_id"],
    right_on=[
        "ufcstats_event_id",
        "bout_order",
        "ufcstats_fighter_1_id",
        "ufcstats_fighter_2_id",
    ],
)[["id", "espn_id"]]
temp_match1 = temp_match1.rename(columns={"id": "ufcstats_id"})

# Do same but swap order of fighters since ESPN doesn't track red/blue corner
espn_bouts2 = espn_bouts.loc[
    ~espn_bouts["espn_id"].isin(temp_match1["espn_id"].values)
].copy()
ufcstats_bouts2 = ufcstats_bouts.loc[
    ~ufcstats_bouts["id"].isin(temp_match1["ufcstats_id"].values)
].copy()
temp_match2 = ufcstats_bouts2.merge(
    espn_bouts2,
    how="inner",
    left_on=["event_id", "bout_order", "red_fighter_id", "blue_fighter_id"],
    right_on=[
        "ufcstats_event_id",
        "bout_order",
        "ufcstats_fighter_2_id",
        "ufcstats_fighter_1_id",
    ],
)[["id", "espn_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

# Repeat first iteration without bout order requirement
espn_bouts3 = espn_bouts2.loc[
    ~espn_bouts2["espn_id"].isin(temp_match2["espn_id"].values)
].copy()
ufcstats_bouts3 = ufcstats_bouts2.loc[
    ~ufcstats_bouts2["id"].isin(temp_match2["ufcstats_id"].values)
].copy()
temp_match3 = ufcstats_bouts3.merge(
    espn_bouts3,
    how="inner",
    left_on=["event_id", "red_fighter_id", "blue_fighter_id"],
    right_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"],
)[["id", "espn_id"]]
temp_match3 = temp_match3.rename(columns={"id": "ufcstats_id"})

# Same but swap fighters
espn_bouts4 = espn_bouts3.loc[
    ~espn_bouts3["espn_id"].isin(temp_match3["espn_id"].values)
].copy()
ufcstats_bouts4 = ufcstats_bouts3.loc[
    ~ufcstats_bouts3["id"].isin(temp_match3["ufcstats_id"].values)
].copy()
temp_match4 = ufcstats_bouts4.merge(
    espn_bouts4,
    how="inner",
    left_on=["event_id", "red_fighter_id", "blue_fighter_id"],
    right_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"],
)[["id", "espn_id"]]
temp_match4 = temp_match4.rename(columns={"id": "ufcstats_id"})

matching2 = pd.concat(
    [temp_match1, temp_match2, temp_match3, temp_match4], ignore_index=True
).reset_index(drop=True)
bout_ids = ufcstats_bouts["id"].values.tolist()

# Sort by ufcstats_id in order to match with ufcstats_bouts
matching2 = matching2.sort_values(
    by="ufcstats_id",
    key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b)),
).reset_index(drop=True)

## FightOdds.io

In [6]:
fightoddsio_bouts = pd.read_csv(
    os.path.join(clean_data_dir, "FightOdds.io", "bouts.csv")
)[["id", "event_id", "fighter_1_id", "fighter_2_id"]]

# Merge with events and bouts mappings
fightoddsio_bouts = fightoddsio_bouts.merge(
    events_linkage[["ufcstats_id", "fightoddsio_id"]].rename(
        columns={"ufcstats_id": "ufcstats_event_id", "fightoddsio_id": "event_id"}
    ),
    how="left",
    on="event_id",
).drop(columns=["event_id"])
fightoddsio_bouts = fightoddsio_bouts.merge(
    fighters_linkage[["ufcstats_id", "fightoddsio_id"]].rename(
        columns={
            "ufcstats_id": "ufcstats_fighter_1_id",
            "fightoddsio_id": "fighter_1_id",
        }
    ),
    how="left",
    on="fighter_1_id",
).drop(columns=["fighter_1_id"])
fightoddsio_bouts = fightoddsio_bouts.merge(
    fighters_linkage[["ufcstats_id", "fightoddsio_id"]].rename(
        columns={
            "ufcstats_id": "ufcstats_fighter_2_id",
            "fightoddsio_id": "fighter_2_id",
        }
    ),
    how="left",
    on="fighter_2_id",
).drop(columns=["fighter_2_id"])
fightoddsio_bouts = fightoddsio_bouts.rename(columns={"id": "fightoddsio_id"})

# FightOdds.io missing first Kazushi Sakuraba vs. Marcus Silveira fight, which ended in NC
ufcstats_bouts_temp = ufcstats_bouts.loc[
    ufcstats_bouts["id"] != "2750ac5854e8b28b"
].copy()

# Match on event ID and fighter IDs
temp_match1 = ufcstats_bouts_temp.merge(
    fightoddsio_bouts,
    how="inner",
    left_on=["event_id", "red_fighter_id", "blue_fighter_id"],
    right_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"],
)[["id", "fightoddsio_id"]]
temp_match1 = temp_match1.rename(columns={"id": "ufcstats_id"})

# Repeat with fighter ID order swapped
fightoddsio_bouts2 = fightoddsio_bouts.loc[
    ~fightoddsio_bouts["fightoddsio_id"].isin(temp_match1["fightoddsio_id"].values)
].copy()
ufcstats_bouts2 = ufcstats_bouts_temp.loc[
    ~ufcstats_bouts_temp["id"].isin(temp_match1["ufcstats_id"].values)
].copy()
temp_match2 = ufcstats_bouts2.merge(
    fightoddsio_bouts2,
    how="inner",
    left_on=["event_id", "red_fighter_id", "blue_fighter_id"],
    right_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"],
)[["id", "fightoddsio_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

# Combine
matching3 = pd.concat([temp_match1, temp_match2], ignore_index=True).reset_index(
    drop=True
)
bout_ids = ufcstats_bouts["id"].values.tolist()

# Sort by ufcstats_id in order to match with ufcstats_bouts
matching3 = matching3.sort_values(
    by="ufcstats_id",
    key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b)),
).reset_index(drop=True)

## MMA Decisions

In [14]:
mmadecisions_bouts = pd.read_csv(
    os.path.join(clean_data_dir, "MMA Decisions", "bouts.csv")
)[["id", "event_id", "fighter_1_id", "fighter_2_id"]]

# Merge with events and bouts mappings
mmadecisions_bouts = mmadecisions_bouts.merge(
    events_linkage[["ufcstats_id", "mmadecisions_id"]].rename(
        columns={"ufcstats_id": "ufcstats_event_id", "mmadecisions_id": "event_id"}
    ),
    how="inner",
    on="event_id",
).drop(columns=["event_id"])
mmadecisions_bouts = mmadecisions_bouts.merge(
    fighters_linkage[["ufcstats_id", "mmadecisions_id"]].rename(
        columns={
            "ufcstats_id": "ufcstats_fighter_1_id",
            "mmadecisions_id": "fighter_1_id",
        }
    ),
    how="left",
    on="fighter_1_id",
).drop(columns=["fighter_1_id"])
mmadecisions_bouts = mmadecisions_bouts.merge(
    fighters_linkage[["ufcstats_id", "mmadecisions_id"]].rename(
        columns={
            "ufcstats_id": "ufcstats_fighter_2_id",
            "mmadecisions_id": "fighter_2_id",
        }
    ),
    how="left",
    on="fighter_2_id",
).drop(columns=["fighter_2_id"])

# Subset on decisions
ufcstats_bouts_temp = (
    ufcstats_bouts.loc[
        (ufcstats_bouts["outcome_method"].str.contains("Decision"))
        | (ufcstats_bouts["outcome_method_details"].str.contains("."))
    ]
    .copy()
    .rename(columns={"id": "ufcstats_id"})
)

# Match on event ID and fighter IDs
temp_match1 = mmadecisions_bouts.merge(
    ufcstats_bouts_temp,
    how="inner",
    left_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"],
    right_on=["event_id", "red_fighter_id", "blue_fighter_id"],
)[["ufcstats_id", "id"]].rename(columns={"id": "mmadecisions_id"})

# Repeat with fighter ID order swapped
mmadecisions_bouts2 = mmadecisions_bouts.loc[
    ~mmadecisions_bouts["id"].isin(temp_match1["mmadecisions_id"].values)
].copy()
temp_match2 = mmadecisions_bouts2.merge(
    ufcstats_bouts_temp,
    how="inner",
    left_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"],
    right_on=["event_id", "red_fighter_id", "blue_fighter_id"],
)[["ufcstats_id", "id"]].rename(columns={"id": "mmadecisions_id"})

# Combine
matching4 = pd.concat([temp_match1, temp_match2], ignore_index=True).reset_index(
    drop=True
)
bout_ids = ufcstats_bouts["id"].values.tolist()

# Sort by ufcstats_id in order to match with ufcstats_bouts
matching4 = matching4.sort_values(
    by="ufcstats_id",
    key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b)),
).reset_index(drop=True)

## Bet MMA

In [15]:
betmma_events = pd.read_csv(os.path.join(clean_data_dir, "Bet MMA", "events.csv"))
event_ids = betmma_events.loc[betmma_events["is_ufc_event"] == 1, "id"].values
betmma_bouts = pd.read_csv(os.path.join(clean_data_dir, "Bet MMA", "bouts.csv"))
betmma_bouts = betmma_bouts.loc[betmma_bouts["event_id"].isin(event_ids)].reset_index(
    drop=True
)

# Merge with events and bouts mappings
betmma_bouts = betmma_bouts.merge(
    events_linkage[["ufcstats_id", "betmma_id"]].rename(
        columns={"ufcstats_id": "ufcstats_event_id", "betmma_id": "event_id"}
    ),
    how="left",
    on="event_id",
).drop(columns=["event_id"])
betmma_bouts = betmma_bouts.merge(
    fighters_linkage[["ufcstats_id", "betmma_id"]].rename(
        columns={"ufcstats_id": "ufcstats_fighter_1_id", "betmma_id": "fighter_1_id"}
    ),
    how="left",
    on="fighter_1_id",
).drop(columns=["fighter_1_id"])
betmma_bouts = betmma_bouts.merge(
    fighters_linkage[["ufcstats_id", "betmma_id"]].rename(
        columns={"ufcstats_id": "ufcstats_fighter_2_id", "betmma_id": "fighter_2_id"}
    ),
    how="left",
    on="fighter_2_id",
).drop(columns=["fighter_2_id"])
betmma_bouts = betmma_bouts.rename(columns={"id": "betmma_id"})

# Match on event ID and fighter IDs
ufcstats_bouts_temp = ufcstats_bouts.loc[
    ufcstats_bouts["event_id"].isin(betmma_bouts["ufcstats_event_id"])
].copy()
temp_match1 = ufcstats_bouts_temp.merge(
    betmma_bouts,
    how="inner",
    left_on=["event_id", "red_fighter_id", "blue_fighter_id"],
    right_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"],
)[["id", "betmma_id"]]
temp_match1 = temp_match1.rename(columns={"id": "ufcstats_id"})

# Repeat with fighter ID order swapped
betmma_bouts2 = betmma_bouts.loc[
    ~betmma_bouts["betmma_id"].isin(temp_match1["betmma_id"].values)
].copy()
ufcstats_bouts2 = ufcstats_bouts_temp.loc[
    ~ufcstats_bouts_temp["id"].isin(temp_match1["ufcstats_id"].values)
].copy()
temp_match2 = ufcstats_bouts2.merge(
    betmma_bouts2,
    how="inner",
    left_on=["event_id", "red_fighter_id", "blue_fighter_id"],
    right_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"],
)[["id", "betmma_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

# Concatenate
matching5 = pd.concat([temp_match1, temp_match2], ignore_index=True).reset_index(
    drop=True
)
bout_ids = ufcstats_bouts["id"].values.tolist()

# Sort by ufcstats_id in order to match with ufcstats_bouts
matching5 = matching5.sort_values(
    by="ufcstats_id",
    key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b)),
).reset_index(drop=True)

## Combine all

In [16]:
matching = matching1.merge(matching2, on="ufcstats_id", how="left")
matching = matching.merge(matching3, on="ufcstats_id", how="left")
matching = matching.merge(matching4, on="ufcstats_id", how="left")
matching = matching.merge(matching5, on="ufcstats_id", how="left")
matching["mmadecisions_id"] = matching["mmadecisions_id"].astype("Int64")
matching["betmma_id"] = matching["betmma_id"].astype("Int64")
matching = matching[
    [
        "ufcstats_id",
        "betmma_id",
        "espn_id",
        "fightoddsio_id",
        "mmadecisions_id",
        "tapology_id",
    ]
]
matching.to_csv(os.path.join(clean_data_dir, "bout_mapping.csv"), index=False)